In [1]:
# get data
import numpy as np
import pandas as pd
# ML stuff
from mne.decoding import CSP
from mne_bids import BIDSPath, read_raw_bids
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
# series to series transformer
from sktime.transformations.series.detrend import Detrender

# panel to panel transformers
from sktime_neuro.transformations.panel.baselinecorrection import \
    BaselineCorrectionTransformer as Baseline
from sktime_neuro.transformations.series.filterforseries import FilterforSeries
from sktime_neuro.transformations.series.seriesdownsampling import \
    SeriesDownsampling
from sktime_neuro.transformations.series_to_panel.eeg_epoching import Epoch
# epoching
from sktime_neuro.utils import mne_processing as utils

## Get Bids Data into sktime Series format

In [2]:
# example can be downloaded here: https://osf.io/cj2dr/
bids_path = BIDSPath(
    subject="08", task="matchingpennies", suffix="eeg", datatype="eeg", root="example"
)

raw = read_raw_bids(bids_path=bids_path, verbose=False)
data = raw.get_data()

Extracting parameters from example/sub-08/eeg/sub-08_task-matchingpennies_eeg.vhdr...
Setting channel info structure...
Reading events from example/sub-08/eeg/sub-08_task-matchingpennies_events.tsv.
The event "left" refers to multiple event values. Creating hierarchical event names.
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left 

    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event: left -> left/1
    Renaming event: left -> left/n/a
    Renaming event: left -> left/n/a
    Renaming event:

    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
  

    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    

    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    Renaming event: right -> right/n/a
    Renaming event: right -> right/2
    Renaming event: right -> right/n/a
    

<ipython-input-2-0fa538297d13>:6: RuntimeWarning: Did not find any eeg.json associated with sub-08_task-matchingpennies.

The search_str was "example/sub-08/**/sub-08*eeg.json"
  raw = read_raw_bids(bids_path=bids_path, verbose=False)


## Apply Series to Series Transformer

In [3]:
downsampled_data = SeriesDownsampling(2).fit_transform(data)
downsampled_data.shape

(10, 4514496)

In [4]:
detrended_data = Detrender().fit_transform(pd.DataFrame(downsampled_data.transpose()))

In [5]:
filtered_data = FilterforSeries(l_freq=8, h_freq=16, sfreq=250).fit_transform(
    detrended_data.transpose()
)

Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 413 samples (1.652 sec)



## Epoch

In [6]:
annotation = utils.create_annotation(raw)

right_MI = Epoch(
    annotation=annotation, label="right/2", interval=[-0.5, 1.5], sfreq=250
).fit_transform(filtered_data)

left_MI = Epoch(
    annotation=annotation, label="left/1", interval=[-0.5, 1.5], sfreq=250
).fit_transform(filtered_data)

## Apply Panel-To-Panel Transformers

In [7]:
right_bl_corrected = Baseline(upper=0.5, fs=250).fit_transform(right_MI)
left_bl_corrected = Baseline(upper=0.5, fs=250).fit_transform(left_MI)

## Classify Trials

In [8]:
data_train = np.concatenate((left_bl_corrected, right_bl_corrected), axis=0)
train_labels = np.concatenate(
    (np.ones(left_bl_corrected.shape[0]), (np.ones(right_bl_corrected.shape[0]) * -1)),
    axis=0,
)


# Assemble a classifier

# filterer = PaneltoPanelFilter() # should not be done for short samples
csp = CSP(transform_into="average_power")
lda = LinearDiscriminantAnalysis()


param_grid = {
    "CSP__n_components": [1, 2, 3, 4, 5, 7, 10],
}

# Use scikit-learn Pipeline with cross_val_score function
pipe = Pipeline(steps=[("CSP", csp), ("LDA", lda)])

search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(data_train, train_labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Computing rank from data with rank=None
    Using tolerance 1.4e-05 (2.2e-16 eps * 10 dim * 6.5e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.3e-05 (2.2e-16 eps * 10 dim * 5.9e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using EMPIRICAL
Done.
Best parameter (CV score=0.590):
{'CSP__n_components': 3}


/home/svea/anaconda3/envs/sktimeneuro2/lib/python3.8/site-packages/mne/decoding/csp.py:184: DeprecationWarning: scipy.linalg.pinv2 is deprecated since SciPy 1.7.0, use scipy.linalg.pinv instead
  self.patterns_ = linalg.pinv2(eigen_vectors)
